In [1]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter
s3 = var("s3", latex_name="s_3")        ## Third relaxation parameter

assume(C,'real')
assume(C>-1/2)
assume(C< 1/2)

s2 = 2
s3 = 2

#### Pieces involved in the scheme
q = 3 ## Number of discrete velocities
c1 = 0  ## First discrete velocity
c2 = 1  ## Second discrete velocity
c3 = -1 ## Third discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1,  1], \
            [0,  1, -1], \
            [0,  1,  1]]) ## Moment matrix
eps1 = 1              ## First equilibrium coefficient (useless)
eps2 = C              ## Second equilibrium coefficient
eps3 = 1/3*(1+2*C**2) ## Third equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa") 

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2, s3])\
        *(matrix([[eps1], [eps2], [eps3]])*matrix([[1, 0, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2), k**(-c3)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z") 

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = (dm1/dp1).full_simplify() ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [2]:
##### Boundary conditions that we analyze for any value of C and s_2 and s_3
bdMat_BB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 1], [0, 0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, -1], [0, 0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [-1, 0, -k], [0, 0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 1, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 2-k, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 3-3*k+k**2, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 4-6*k+4*k**2-k**3, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 0], [0, 0, k]])*Minv*K ## Kinetic dirichlet

In [3]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k))) 
print("Two steps anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [4]:
##### Sanity check : we set some values for C and see if we find more
##### Sometimes sagemath does not give all the roots

C_val = -3/13

print("Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_BB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_ABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots")
print("Two steps anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_TwoABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma1.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma2.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma3.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma4.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_kinDir.subs(C=C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1], [z == -16/507*I*sqrt(935) + 133/507, kappa == -1872/73529*I*sqrt(935) - 46151/73529], [z == 16/507*I*sqrt(935) + 133/507, kappa == 1872/73529*I*sqrt(935) - 46151/73529]]

!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots
Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [75]:
print("Anti Bounce back (now correct)")
kABB = solve(bdMat_ABB.determinant().full_simplify(), k)[0].rhs().full_simplify()
zABB = solve(charEq.full_simplify().subs(k==kABB), z)
pretty_print(zABB)

zABBMinus = zABB[0].rhs()
zABBPlus  = zABB[1].rhs()

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBMinus).full_simplify(), k))

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBPlus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBPlus).full_simplify(), k))

#print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))

R = -(4*C^4 - 2*C^2 - 2).factor()
RealPoly = 32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - 6*C + 10
ImagPoly = 16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2
ER = (1/2*(RealPoly+RealPoly.subs(C==-C))).factor()
OR = (1/2*(RealPoly-RealPoly.subs(C==-C))).factor()
EI = (1/2*(ImagPoly+ImagPoly.subs(C==-C))).factor()
OI = (1/2*(ImagPoly-ImagPoly.subs(C==-C))).factor()

print('values of E_R, O_R, E_I and O_I')
pretty_print(ER)
pretty_print(OR)
pretty_print(EI)
pretty_print(OI)

er = var("er", latex_name="\\mathcal{E}_R") 
ei = var("ei", latex_name="\\mathcal{E}_I") 
odr = var("odr", latex_name="\\mathcal{O}_R") 
odi = var("odi", latex_name="\\mathcal{O}_I") 
r = var("r", latex_name="\\mathcal{R}") 
assume(er, 'real')
assume(ei, 'real')
assume(odr, 'real')
assume(odi, 'real')
assume(r>0)


num = (er+odr+I*(ei+odi)*sqrt(r))
den = (er-odr+I*(ei-odi)*sqrt(r))

numMod = ((num*conjugate(den)).full_simplify())
denMod = ((den*conjugate(den)).full_simplify())
print('Numerator in abs(k)^2')
pretty_print((numMod.real()**2+numMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())
print('Denominator in abs(k)^2')
pretty_print((denMod.real()**2+denMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())

Anti Bounce back (now correct)


[z == -4/3*C^2 - 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -4/3*C^2 + 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10), kappa == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 - (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

...

3*(8*C^2 - 5)*C

Numerator in abs(k)^2


2916*(4*C^4 - 11*C^2 - 2)^2*(C + 1)^2*(C - 1)^2

Denominator in abs(k)^2


2916*(4*C^4 - 11*C^2 - 2)^2*(C + 1)^2*(C - 1)^2